In [292]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

In [293]:
# Load raw CSV
df = pd.read_csv("ahmedabad.csv")  # update filename if different


In [294]:
def parse_currency(x):
    if pd.isnull(x):
        return np.nan
    if isinstance(x, (int,float,np.number)):
        try:
            return float(x)
        except:
            return np.nan
    s = str(x).strip()
    s = s.replace('₹','').replace('Rs.','').replace('Rs','').replace(',','').strip()
    m_cr = re.search(r'([0-9]*\.?[0-9]+)\s*(cr|Cr|CR|crore|Crore)', s)
    if m_cr:
        val = float(m_cr.group(1))
        return val * 1e7
    m_lac = re.search(r'([0-9]*\.?[0-9]+)\s*(l|L|lac|Lac|lakh|Lakh)', s)
    if m_lac:
        val = float(m_lac.group(1))
        return val * 1e5
    m_k = re.search(r'([0-9]*\.?[0-9]+)\s*(k|K)', s)
    if m_k:
        val = float(m_k.group(1))
        return val * 1e3
    m_num = re.search(r'[-+]?[0-9]*\,?[0-9]+(?:\.[0-9]+)?', s.replace(',',''))
    if m_num:
        try:
            return float(m_num.group(0))
        except:
            return np.nan
    return np.nan

In [295]:
#clearning price and price_sqft
df['price'] = df['price'].apply(parse_currency)
df['price_sqft'] = df['price_sqft'].apply(parse_currency)

In [296]:
def area_to_sqft(x):
    if pd.isna(x):
        return np.nan
    x = str(x).lower().strip()
    # extract numeric value
    match = re.search(r'\d+(\.\d+)?', x)
    if not match:
        return np.nan
    try:
        value = float(match.group())
    except ValueError:
        return np.nan
    if 'sqyrd' in x:
        return value * 9  # 1 sqyrd = 9 sqft
    elif 'sqft' in x:
        return value
    else:
        return value

In [297]:
df['value_area_sqft'] = df['value_area'].apply(area_to_sqft)

In [298]:
current_year = 2025  # adjust if needed

def extract_possession_year(status):
    """Extract numeric possession year from status."""
    if pd.isna(status):
        return np.nan
    status = str(status)
    match = re.search(r"'(\d{2})", status)
    if match:
        year = int(match.group(1))
        return 2000 + year  # convert '26 → 2026
    elif "Ready to Move" in status:
        return current_year
    else:
        return np.nan

In [299]:
# Apply extraction
df['possession_year'] = df['status'].apply(extract_possession_year)

# Binary: ready to move
df['ready_to_move'] = df['status'].apply(lambda x: 1 if 'Ready' in str(x) else 0)

# Years until possession
df['years_until_possession'] = df['possession_year'] - current_year



In [300]:
def extract_bhk(title):
    title = str(title).lower()
    if 'studio' in title:
        return 'Studio'
    match = re.search(r'(\d+)\s*bhk', title)
    if match:
        return int(match.group(1))
    return None

df['bhk'] = df['Title'].apply(extract_bhk)


def extract_project(title):
    title = str(title)
    match = re.search(r'in (.*?),', title)
    if match:
        return match.group(1).strip()
    return None

df['project_name'] = df['Title'].apply(extract_project)



def extract_locality(title):
    title = str(title)
    parts = title.split(',')
    if len(parts) >= 2:
        return parts[-1].strip()  # last part = locality
    return None

df['locality'] = df['Title'].apply(extract_locality)



def extract_floor_type(title):
    title = str(title).lower()
    if 'basement' in title:
        return 'Basement'
    if 'builder floor' in title:
        return 'Builder Floor'
    return 'Regular'

df['floor_type'] = df['Title'].apply(extract_floor_type)


In [301]:
def extract_floor_info(floor):
    if pd.isna(floor):
        return pd.Series([np.nan, np.nan])
    
    floor = str(floor).lower().strip()
    
    # Check for "ground"
    floor = floor.replace("ground", "0")
    
    # Match pattern "X out of Y"
    match = re.match(r'(\d+)\s*out of\s*(\d+)', floor)
    if match:
        floor_num = int(match.group(1))
        total_floors = int(match.group(2))
        return pd.Series([floor_num, total_floors])
    
    # If not a floor, return NaN
    return pd.Series([np.nan, np.nan])

df[['floor_num', 'total_floors']] = df['floor'].apply(extract_floor_info)



def extract_transaction(floor):
    if pd.isna(floor):
        return np.nan
    floor = str(floor).lower()
    if 'new property' in floor:
        return 'New Property'
    elif 'resale' in floor:
        return 'Resale'
    else:
        return 'Other'

df['transaction'] = df['floor'].apply(extract_transaction)



In [302]:
df['transaction'] = df['transaction'].apply(lambda x: str(x).strip())
df['transaction'] = df['transaction'].replace({'Unfurnished': 'Unknown'})  # fix misaligned rows
df['transaction_code'] = df['transaction'].astype('category').cat.codes  # numeric encoding

In [303]:
df['furnishing'] = df['furnishing'].apply(lambda x: str(x).strip())
df['furnishing_code'] = df['furnishing'].astype('category').cat.codes

In [304]:
# Step 1: Clean facing column
df[['facing_dir', 'extra_features']] = df['facing'].apply(lambda x: pd.Series(clean_facing(x)))

# Step 2: Map directions to numeric angles
direction_map = {
    "North": 0,
    "North-East": 45,
    "East": 90,
    "South-East": 135,
    "South": 180,
    "South-West": 225,
    "West": 270,
    "North-West": 315
}
df['facing_angle'] = df['facing_dir'].map(direction_map).fillna(-1)  # -1 for unknown

# Step 3: One-hot encode directions
for d in direction_map.keys():
    df[f'facing_{d}'] = df['facing_dir'].apply(lambda x: 1 if x == d else 0)

# Step 4: One-hot encode extra features
for feat in ['Garden/Park', 'Pool', 'Main Road']:
    df[f'extra_{feat}'] = df['extra_features'].apply(lambda x: 1 if feat in x else 0)


In [305]:
df['bhk'] = df['bhk'].replace({'Studio': 1}).astype(float)



le_type_area = LabelEncoder()
df['type_area'] = le_type_area.fit_transform(df['type_area'].fillna('Unknown'))

for col in ['project_name', 'locality', 'floor_type']:
    df[col] = df[col].fillna('Unknown')
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


C:\Users\LEGION\AppData\Local\Temp\ipykernel_25460\122503190.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['bhk'] = df['bhk'].replace({'Studio': 1}).astype(float)


In [306]:
dfdrop_cols = [
    'Unnamed: 0', 'facing', 'facing_dir', 'extra_features', 
    'Title', 'value_area', 'floor', 'furnishing', 'transaction', 'status', 'description', 
]

df= df.drop(columns=drop_cols)


In [307]:
### filling or removing the empty ###

In [308]:
df = df.dropna(subset=['price', 'price_sqft'])
df = df[(df['price'] > 0) & (df['price_sqft'] > 0)]


In [309]:
for col in ['floor_num', 'total_floors', 'price_sqft']:
    df[col] = df[col].fillna(df[col].median())


In [310]:
# Fill numeric columns with median
df['possession_year'] = df['possession_year'].fillna(df['possession_year'].median())
df['years_until_possession'] = df['years_until_possession'].fillna(df['years_until_possession'].median())
df['bhk'] = df['bhk'].replace('Studio', 1)  # convert 'Studio' to 1
df['bhk'] = df['bhk'].fillna(df['bhk'].median())


In [313]:
df.columns

Index(['type_area', 'price', 'price_sqft', 'value_area_sqft',
       'possession_year', 'ready_to_move', 'years_until_possession', 'bhk',
       'project_name', 'locality', 'floor_type', 'floor_num', 'total_floors',
       'transaction_code', 'furnishing_code', 'facing_angle', 'facing_North',
       'facing_North-East', 'facing_East', 'facing_South-East', 'facing_South',
       'facing_South-West', 'facing_West', 'facing_North-West',
       'extra_Garden/Park', 'extra_Pool', 'extra_Main Road'],
      dtype='object')

In [314]:
df.to_csv('cleaned_data.csv', index=False)